In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import joblib
import tensorflow as tf
#%tensorflow_version 1.x

In [ ]:
data = pd.read_csv('/content/Training.csv')
data.head()

,Disease_name,"Abdomen, Acute",Abdominal Pain,Abdominal bloating,Abdominal colic,Abducens Nerve Diseases,Abnormal bowel sounds,Abnormal breathing,Abnormal heart beat,Abnormal sputum,...,urinoma,verbal auditory hallucinations,verbally abusive behavior,vision blurred,weepiness,welt,wheelchair bound,withdraw,worry,yellow sputum
0,Achalasia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acromegaly,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Amyloidosis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ascites,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = data[data.columns[1:]]
y = data['Disease_name']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 21)

In [ ]:
#@title Training { output-height: 40, form-width: "40%" }
### Training
clf = RandomForestClassifier(n_estimators=80,
                             criterion = 'entropy',
                             max_depth = 45,
                             min_samples_split = 3,
                             min_samples_leaf = 2,
                             max_features = None,
                             min_impurity_decrease = 0.01,
                             random_state = 21
                             )
clf.fit(X,y)

RandomForestClassifier(criterion='entropy', max_depth=45, max_features=None,
                       min_impurity_decrease=0.01, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=80, random_state=21)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = clf.predict(X)

print(f'Accuracy: {accuracy_score(y, y_pred)}')

Accuracy: 0.8819095477386935


In [ ]:
import pickle

with open('RFClassifier.pkl','wb') as rf:
    pickle.dump(clf, rf)

with open('RFClassifier.pkl', 'rb') as f:
    rf = pickle.load(f)

y_pred = rf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.8666666666666667


In [ ]:
import joblib
joblib.dump(clf, 'RFClassifier_disease_predict.pkl')

['RFClassifier_disease_predict.pkl']

In [ ]:
y_prob = clf.predict_proba(X_test)[:,1]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

features = data.columns[1:]

In [ ]:
Fea_Dict = {}
for i,f in enumerate(features):
    Fea_Dict[f] = i

In [ ]:
search = ['Diarrhea','Nausea','Vomiting','Chills','Fever','Headache']

sample = np.zeros((len(features),), dtype=np.int)

for i,s in enumerate(search):
    sample[Fea_Dict[s]] = 1

sample_in = np.array(sample).reshape(1,len(sample))

rf.predict(sample_in)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array(['malaria'], dtype=object)

In [ ]:
c  = pd.DataFrame(rf.predict_proba(sample_in), columns=rf.classes_)
c = c.sort_values(axis=1, by= 0, ascending= False)
c

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


,malaria,chronic renal insufficiency,pyelonephritis,cerebral venous thrombosis,meningitis,thyreotoxic crisis,q - fever,systemic lupus erythematosus,migraene,bacteremia,...,dependence,dementia,delusion,delirium,deglutition disorder,degenerative polyarthritis,deep vein thrombosis,decubitus ulcer,cushing - syndrome,zinkmangel
0,0.160476,0.145655,0.11372,0.068452,0.06625,0.062292,0.049583,0.032113,0.029167,0.028393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
first_n_column  = c.iloc[: , :10]
first_n_column

,malaria,chronic renal insufficiency,pyelonephritis,cerebral venous thrombosis,meningitis,thyreotoxic crisis,q - fever,systemic lupus erythematosus,migraene,bacteremia
0,0.160476,0.145655,0.11372,0.068452,0.06625,0.062292,0.049583,0.032113,0.029167,0.028393


# Result to return

In [ ]:
d = first_n_column.to_dict('split')
d

{'index': [0],
 'columns': ['malaria',
  'chronic renal insufficiency',
  'pyelonephritis',
  'cerebral venous thrombosis',
  'meningitis',
  'thyreotoxic crisis',
  'q - fever',
  'systemic lupus erythematosus',
  'migraene',
  'bacteremia'],
 'data': [[0.16047619047619052,
   0.1456547619047619,
   0.1137202380952381,
   0.06845238095238096,
   0.06625,
   0.06229166666666667,
   0.049583333333333326,
   0.03211309523809524,
   0.029166666666666664,
   0.028392857142857143]]}

In [ ]:
disease_arr = d['columns']
disease_arr

['malaria',
 'chronic renal insufficiency',
 'pyelonephritis',
 'cerebral venous thrombosis',
 'meningitis',
 'thyreotoxic crisis',
 'q - fever',
 'systemic lupus erythematosus',
 'migraene',
 'bacteremia']

In [ ]:
prob_arr = []
for i in d['data'][0]:
  prob_arr.append(i*100)

In [ ]:
prob_arr

[16.04761904761905,
 14.56547619047619,
 11.37202380952381,
 6.845238095238096,
 6.625,
 6.229166666666667,
 4.958333333333333,
 3.2113095238095237,
 2.9166666666666665,
 2.8392857142857144]